### Library Imports
Importing Necessary Libraries Here

In [30]:
# Purpose : Data Manipulation
import pandas as pd
import numpy as np

# Purpose : Visualization
import seaborn as sns
import matplotlib.pyplot as plt

In [31]:
# Loading in Datasets
season_df = pd.read_csv('l1_final.csv')
match_df = pd.read_csv('l1_match.csv')

In [32]:
# Looking at the Datasets
season_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 23 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Rk               100 non-null    int64  
 1   Squad            100 non-null    object 
 2   MP               100 non-null    int64  
 3   W                100 non-null    int64  
 4   D                100 non-null    int64  
 5   L                100 non-null    int64  
 6   GF               100 non-null    int64  
 7   GA               100 non-null    int64  
 8   GD               100 non-null    int64  
 9   Pts              100 non-null    int64  
 10  Pts/MP           100 non-null    float64
 11  xG               100 non-null    float64
 12  xGA              100 non-null    float64
 13  xGD              100 non-null    float64
 14  xGD/90           100 non-null    float64
 15  Attendance       83 non-null     object 
 16  SSN              100 non-null    object 
 17  Age              

Columns with Missing Values: Attendance, Age

In [33]:
# Looking atthe Datasets
match_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2038 entries, 0 to 2037
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Wk          1900 non-null   float64
 1   Home        1904 non-null   object 
 2   xG          1799 non-null   float64
 3   Score       1803 non-null   object 
 4   xG.1        1799 non-null   float64
 5   Away        1904 non-null   object 
 6   Attendance  1788 non-null   object 
 7   Venue       1904 non-null   object 
 8   SSN         2038 non-null   object 
 9   xG_Winner   2038 non-null   object 
 10  Home_Score  1803 non-null   object 
 11  Away_Score  1803 non-null   object 
 12  obs_Winner  2038 non-null   object 
dtypes: float64(3), object(10)
memory usage: 207.1+ KB


Columns with Missing Values: Attendance

In [34]:
# Filling in Missing Age Values
mean_value = str(pd.to_numeric(l1["Age"]).mean())
season_df["Age"].fillna(mean_value, inplace=True)

In [35]:
# Applying pd.to_numeric to Attendance for both dataframes + Age for Season DataFrame
match_df["Attendance"] = match_df["Attendance"].str.replace(',', '')
match_df["Attendance"] = match_df["Attendance"].apply(pd.to_numeric, errors = 'coerce', downcast = 'float')

change_cols = ["Attendance", "Age"]
season_df["Attendance"] = season_df["Attendance"].str.replace(',', '')
season_df[change_cols] = season_df[change_cols].apply(pd.to_numeric, errors = 'coerce', downcast = 'float')

In [36]:
# Checking Data Types of Match DF
s = (season_df.dtypes == 'object')
object_cols = list(s[s].index)
object_cols

['Squad', 'SSN']

In [37]:
# Checking Data Types for Seasons DF
s = (match_df.dtypes == 'object')
object_cols = list(s[s].index)
object_cols

['Home',
 'Score',
 'Away',
 'Venue',
 'SSN',
 'xG_Winner',
 'Home_Score',
 'Away_Score',
 'obs_Winner']

In [38]:
# Sorting Dataframes to be chronologically ordered
season_df.sort_values(by = ["SSN"], inplace=True)
match_df.sort_values(by=["SSN", "Wk"], inplace=True)

In [39]:
# Selecting Columns from Seasons DF to merge into Match Df
col_needed = ['Prev_Rk', 'Age', 'Squad', 'SSN', 'Value', 'Offense_Rating', 'Midfield_Rating', 'Defense_Rating']
season_df1 = season_df[col_needed]

In [41]:
# Merging Dataframes
df = match_df.merge(season_df1, left_on=['Home', 'SSN'], right_on=['Squad', 'SSN'], how = 'left')
df = df.rename(columns={'Prev_Rk':'Home_Team_Prev_Rk', 'Age': 'Home_Team_Age', 'Value':'Home_Team_Value', 'Offense_Rating':'Home_Team_Offense_Rating', 'Midfield_Rating':'Home_Team_Midfield_Rating', 'Defense_Rating':'Home_Team_Defense_Rating'})
df = df.merge(season_df1, left_on=['Away', 'SSN'], right_on=['Squad', 'SSN'], how = 'left')
df = df.rename(columns={'Prev_Rk':'Away_Team_Prev_Rk', 'Age': 'Away_Team_Age', 'Value':'Away_Team_Value', 'Offense_Rating':'Away_Team_Offense_Rating', 'Midfield_Rating':'Away_Team_Midfield_Rating', 'Defense_Rating':'Away_Team_Defense_Rating'})
df.columns
df.drop(columns=["Squad_x", "Squad_y"],inplace=True)
df.sort_values(by=["SSN", "Wk"], inplace=True)
df.reset_index(drop=True, inplace=True)
df.drop(columns=['Venue', 'xG_Winner'], inplace=True)

In [58]:
# Making Copy of Dataframe
ligue_un = df.copy()

In [59]:
# Making Match Point Results Function for each Match
def home_result() :
    conditions = [ligue_un["obs_Winner"] == 'Home', ligue_un["obs_Winner"] == 'Draw', ligue_un["obs_Winner"] == 'Away']
    choices = [3, 1, 0]
    ligue_un['Home_Result'] = np.select(conditions, choices)


def away_result() :
    conditions = [ligue_un["obs_Winner"] == 'Home', ligue_un["obs_Winner"] == 'Draw', ligue_un["obs_Winner"] == 'Away']
    choices = [0, 1, 3]
    ligue_un['Away_Result'] = np.select(conditions, choices)

def result_maker() :
    home_result()
    away_result()
    return ligue_un

In [60]:
# Applying result_maker() function to dataframe
result_maker
ligue_un


,Wk,Home,xG,Score,xG.1,Away,Attendance,SSN,Home_Score,Away_Score,...,Home_Team_Value,Home_Team_Offense_Rating,Home_Team_Midfield_Rating,Home_Team_Defense_Rating,Away_Team_Prev_Rk,Away_Team_Age,Away_Team_Value,Away_Team_Offense_Rating,Away_Team_Midfield_Rating,Away_Team_Defense_Rating
0,1.0,Monaco,2.0,3–2,0.3,Toulouse,13572.0,2017-2018,3,2,...,267.40,80.0,81.0,78.0,13.0,26.600000,73.45,73.0,74.0,74.0
1,1.0,Paris S-G,2.6,2–0,0.3,Amiens,46898.0,2017-2018,2,0,...,648.40,86.0,82.0,82.0,21.0,26.400000,21.15,75.0,73.0,72.0
2,1.0,Lyon,1.8,4–0,0.5,Strasbourg,1979.0,2017-2018,4,0,...,3149.55,80.0,78.0,77.0,21.0,25.000000,94.60,70.0,73.0,71.0
3,1.0,Saint-Étienne,0.9,1–0,0.5,Nice,25879.0,2017-2018,1,0,...,42.00,75.0,75.0,76.0,3.0,22.600000,143.65,78.0,77.0,76.0
4,1.0,Metz,0.6,1–3,1.9,Guingamp,14595.0,2017-2018,1,3,...,25.95,69.0,73.0,72.0,10.0,25.799999,17.40,78.0,74.0,73.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2033,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-2022,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2034,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-2022,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2035,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-2022,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2036,NaN,Auxerre,NaN,1–1,NaN,Saint-Étienne,17479.0,2021-2022,1,1,...,NaN,NaN,NaN,NaN,11.0,25.299999,42.00,75.0,73.0,72.0


In [61]:
ligue_un.columns

Index(['Wk', 'Home', 'xG', 'Score', 'xG.1', 'Away', 'Attendance', 'SSN',
       'Home_Score', 'Away_Score', 'obs_Winner', 'Home_Team_Prev_Rk',
       'Home_Team_Age', 'Home_Team_Value', 'Home_Team_Offense_Rating',
       'Home_Team_Midfield_Rating', 'Home_Team_Defense_Rating',
       'Away_Team_Prev_Rk', 'Away_Team_Age', 'Away_Team_Value',
       'Away_Team_Offense_Rating', 'Away_Team_Midfield_Rating',
       'Away_Team_Defense_Rating'],
      dtype='object')